## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `parentRAG.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [ ]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

In [5]:
# Carregar modelos Open AI - Embedding e chat

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    max_tokens=500,
)

In [6]:
# Carregar o PDF
pdf_link = "DOC-SF238339076816-20230503.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()

In [ ]:
len(pages)

In [8]:
# Separar em chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True
)

chunks = text_splitter.split_documents(pages)

In [9]:
# Salvar os Chunks no VectorDB

vectordb = Chroma.from_documents(chunks, embedding=embeddings, persist_directory="naiveDB")

In [12]:
# Carregar o DB

naive_retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [13]:
# https://dashboard.cohere.com/api-keys
rerank = CohereRerank(model="rerank-v3.5", top_n=3)

compressor_retriever = ContextualCompressionRetriever(
    base_compressor=rerank,
    base_retriever=naive_retriever,
)


In [14]:
TEMPLATE = """
    Você é um especialista em legislação e tecnologia. Responda a pergunta abaixo utilizando o contexto informado.
    Query:
    {question}
    Context:
    {context}
"""
rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [15]:
setup_retrival = RunnableParallel({"question": RunnablePassthrough(), "context": compressor_retriever})
output_parser = StrOutputParser()

compressor_retrieval_chain = setup_retrival | rag_prompt | llm | output_parser

In [ ]:
compressor_retrieval_chain.invoke("Quais os principais riscos do marco legal de ia")